In [2]:
!pip install librosa sounddevice soundfile scikit-learn


In [3]:
import os
import numpy as np
import librosa
import sounddevice as sd
import soundfile as sf

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier


In [4]:
moods = ["calm", "neutral", "energetic", "angry"]

for mood in moods:
    os.makedirs(f"mood_dataset/{mood}", exist_ok=True)

print("Dataset folders created")


Dataset folders created


In [5]:
def record_sample(mood, idx, duration=3, sr=22050):
    print(f"Recording {mood.upper()} sample {idx}")
    audio = sd.rec(int(duration * sr), samplerate=sr, channels=1)
    sd.wait()
    sf.write(f"mood_dataset/{mood}/{mood}_{idx}.wav", audio, sr)


In [6]:
for i in range(5):   # 5 per mood is enough
    record_sample("calm", i)
    record_sample("neutral", i)
    record_sample("energetic", i)
    record_sample("angry", i)


Recording CALM sample 0
Recording NEUTRAL sample 0
Recording ENERGETIC sample 0
Recording ANGRY sample 0
Recording CALM sample 1
Recording NEUTRAL sample 1
Recording ENERGETIC sample 1
Recording ANGRY sample 1
Recording CALM sample 2
Recording NEUTRAL sample 2
Recording ENERGETIC sample 2
Recording ANGRY sample 2
Recording CALM sample 3
Recording NEUTRAL sample 3
Recording ENERGETIC sample 3
Recording ANGRY sample 3
Recording CALM sample 4
Recording NEUTRAL sample 4
Recording ENERGETIC sample 4
Recording ANGRY sample 4


In [8]:
def extract_features(file):
    audio, sr = librosa.load(file, sr=22050)

    mfcc = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13), axis=1)
    zcr = np.mean(librosa.feature.zero_crossing_rate(audio))
    rms = np.mean(librosa.feature.rms(y=audio))

    return np.hstack([mfcc, zcr, rms])


In [9]:
X = []
y = []

label_map = {
    "calm": 0,
    "neutral": 1,
    "energetic": 2,
    "angry": 3
}

for mood, label in label_map.items():
    folder = f"mood_dataset/{mood}"
    for file in os.listdir(folder):
        features = extract_features(f"{folder}/{file}")
        X.append(features)
        y.append(label)

X = np.array(X)
y = np.array(y)

print("Dataset shape:", X.shape)


C:\Users\Dell\anaconda3\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\Dell\anaconda3\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
C:\Users\Dell\anaconda3\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


Dataset shape: (20, 15)


In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = MLPClassifier(
    hidden_layer_sizes=(64, 32),
    activation='relu',
    max_iter=500
)

model.fit(X_train, y_train)
print("Model trained successfully")


Model trained successfully


C:\Users\Dell\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


In [11]:
print("Accuracy:", model.score(X_test, y_test))


Accuracy: 0.0


In [12]:
def mood_meter():
    print("Speak now...")
    audio = sd.rec(int(3 * 22050), samplerate=22050, channels=1)
    sd.wait()
    sf.write("temp.wav", audio, 22050)

    features = extract_features("temp.wav").reshape(1, -1)
    mood = model.predict(features)[0]

    messages = {
        0: "😴 Very Calm — you sound relaxed",
        1: "😐 Neutral — balanced vibes",
        2: "😄 Energetic — full of energy!",
        3: "😡 Stressed — take a deep breath"
    }

    print(messages[mood])

mood_meter()


Speak now...
😐 Neutral — balanced vibes
